# Text Translation and Sentiment Analysis using Transformers


## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [9]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [10]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

import pandas as pd

# Column mapping for standardization
COLUMN_MAPPING = {
    'Movie / TV Series': 'Title',
    'Título': 'Title',
    'Titre': 'Title',
    'Année': 'Year',
    'Año': 'Year',
    'Synopsis': 'Synopsis',
    'Sinopsis': 'Synopsis',
    'Critiques': 'Review',
    'Críticas': 'Review'
}

def process_file(file_path: str, language: str) -> pd.DataFrame:
    """
    Helper function to read and process a CSV file by mapping columns
    and adding the "Original Language" column.
    """
    df = pd.read_csv(file_path)
    df = df.rename(columns=COLUMN_MAPPING)
    df["Original Language"] = language
    return df[['Title', 'Year', 'Synopsis', 'Review', 'Original Language']]

def preprocess_data() -> pd.DataFrame:
    """
    Preprocess and combine movie data from English, Spanish, and French CSV files.
    """
    files = [
        ("data/movie_reviews_eng.csv", "English"),
        ("data/movie_reviews_sp.csv", "Spanish"),
        ("data/movie_reviews_fr.csv", "French")
    ]
    
    # Process each file and concatenate
    df_list = [process_file(file, lang) for file, lang in files]
    df = pd.concat(df_list, ignore_index=True)
    
    return df

df = preprocess_data()

In [11]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
12,Y tu mamá también,2001,Dos amigos adolescentes (Gael García Bernal y ...,"""Y tu mamá también es una película que se qued...",Spanish
18,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",Spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
25,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",French
24,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
27,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",French
13,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",Spanish
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
28,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [12]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from huggingface_hub import login
from pathlib import Path
import os

# Define home directory and token file path
HOME = Path.home()
TOKEN_FILE = HOME / ".hftoken"

# Authenticate with Hugging Face
def authenticate_huggingface(token_path: Path):
    try:
        with open(token_path, "r") as file:
            hf_token = file.read().strip()
        login(token=hf_token)
        print("Successfully logged into Hugging Face.")
    except FileNotFoundError:
        print(f"Token file not found at {token_path}. Please ensure the file exists.")
    except Exception as e:
        print(f"Error during authentication: {e}")

# Authenticate
authenticate_huggingface(TOKEN_FILE)

# Model configuration
MODEL_CACHE_DIR = HOME / ".cache/huggingface/hub"
MODELS = {
    "fr_en": "Helsinki-NLP/opus-mt-fr-en",
    "es_en": "Helsinki-NLP/opus-mt-es-en"
}
CACHE_PATHS = {
    "fr_en": MODEL_CACHE_DIR / "models--Helsinki-NLP--opus-mt-fr-en",
    "es_en": MODEL_CACHE_DIR / "models--Helsinki-NLP--opus-mt-es-en"
}

# Load models and tokenizers
def load_model_and_tokenizer(model_name: str, cache_path: Path):
    try:
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name, local_files_only=True
        )
        tokenizer = AutoTokenizer.from_pretrained(
            model_name, local_files_only=True
        )
        print(f"Loaded {model_name} from cache.")
    except Exception:
        print(f"{model_name} not found in cache. Downloading...")
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

fr_en_model, fr_en_tokenizer = load_model_and_tokenizer(
    MODELS["fr_en"], CACHE_PATHS["fr_en"]
)
es_en_model, es_en_tokenizer = load_model_and_tokenizer(
    MODELS["es_en"], CACHE_PATHS["es_en"]
)

# Translation function
def translate(text: str, model, tokenizer) -> str:
    """
    Translate text using the specified model and tokenizer.
    """
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Specific translation functions
def translate_fr_en(text: str) -> str:
    return translate(text, fr_en_model, fr_en_tokenizer)

def translate_es_en(text: str) -> str:
    return translate(text, es_en_model, es_en_tokenizer)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/ewkoch/.cache/huggingface/token
Login successful
Successfully logged into Hugging Face.


/Users/ewkoch/repos/udacity-deep-learning-4-transformers/.conda/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Loaded Helsinki-NLP/opus-mt-fr-en from cache.
Loaded Helsinki-NLP/opus-mt-es-en from cache.


In [13]:
# Filter reviews and synopsis by language
fr_reviews = df[df['Original Language'] == 'French']['Review']
fr_synopsis = df[df['Original Language'] == 'French']['Synopsis']

es_reviews = df[df['Original Language'] == 'Spanish']['Review']
es_synopsis = df[df['Original Language'] == 'Spanish']['Synopsis']

# Translate French reviews and synopsis to English
fr_reviews_en = fr_reviews.apply(lambda x: translate_fr_en(x) if pd.notnull(x) else x)
fr_synopsis_en = fr_synopsis.apply(lambda x: translate_fr_en(x) if pd.notnull(x) else x)

# Translate Spanish reviews and synopsis to English
es_reviews_en = es_reviews.apply(lambda x: translate_es_en(x) if pd.notnull(x) else x)
es_synopsis_en = es_synopsis.apply(lambda x: translate_es_en(x) if pd.notnull(x) else x)

# Update DataFrame by overwriting the original columns or adding new ones
df.loc[df['Original Language'] == 'French', 'Review'] = fr_reviews_en
df.loc[df['Original Language'] == 'French', 'Synopsis'] = fr_synopsis_en

df.loc[df['Original Language'] == 'Spanish', 'Review'] = es_reviews_en
df.loc[df['Original Language'] == 'Spanish', 'Synopsis'] = es_synopsis_en


print("Translation complete. DataFrame updated.")

Translation complete. DataFrame updated.


In [14]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English
22,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French
23,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French
25,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",French
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
26,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",French
18,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish
11,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish
13,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load sentiment analysis model and tokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name)
sentiment_tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to analyze sentiment
def analyze_sentiment(text, model, tokenizer):
    """
    Perform sentiment analysis on text using a model and tokenizer.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    
    # Get the predicted class (logits to class)
    predicted_class = torch.argmax(outputs.logits).item()
    
    # Map class to Positive or Negative sentiment
    sentiment = "Positive" if predicted_class > 2 else "Negative"
    return sentiment

In [19]:
df['Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_model, sentiment_tokenizer))

In [20]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
20,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",French,Positive
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English,Negative
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English,Negative
18,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish,Negative
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,Positive
23,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,Positive
16,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish,Negative
11,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish,Positive
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English,Negative
28,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French,Negative


In [22]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)